### It happens all the time: someone gives you data containing malformed strings,Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

- Take this monstrosity as the DataFrame to use in the following puzzles:


In [1]:
import pandas as pd

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']})


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   From_To       5 non-null      object 
 1   FlightNumber  3 non-null      float64
 2   RecentDelays  5 non-null      object 
 3   Airline       5 non-null      object 
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes


### 1. Some values in the the FlightNumber column are missing. 
####  These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. 
#### Fill in these missing numbers and make the column an integer column (instead of a float column).

In [4]:
FlightNumber_colindex = 1

for i in range(df.shape[0]):
    if np.isnan(df['FlightNumber'].iloc[i]):
        df.iloc[i, FlightNumber_colindex] = 10045.0 + i * 10
        

df['FlightNumber'] = df['FlightNumber'].astype(int)
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   From_To       5 non-null      object
 1   FlightNumber  5 non-null      int32 
 2   RecentDelays  5 non-null      object
 3   Airline       5 non-null      object
dtypes: int32(1), object(3)
memory usage: 268.0+ bytes


### 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [6]:
temp_df = df['From_To'].str.split('_', n = 1, expand = True) 

In [7]:
temp_df

,0,1
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [8]:
temp_df.columns = ['From', 'To']

In [9]:
temp_df

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


### 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [10]:
temp_df['From'] = temp_df['From'].str.capitalize()
temp_df['To'] = temp_df['To'].str.capitalize()

In [11]:
temp_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


### 4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [12]:
df.drop('From_To', axis=1, inplace=True)

In [13]:
df = pd.concat([df, temp_df] ,axis=1)

In [14]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


### 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. 
- We would like each first value in its own column, each second value in its own column, and so on. 
-  If there isn't an Nth value, the value should be NaN.

In [15]:
df['RecentDelays']

0        [23, 47]
1              []
2    [24, 43, 87]
3            [13]
4        [67, 32]
Name: RecentDelays, dtype: object

In [16]:
#Determine the maximum number of elements in all the RecentDelays list
lambda_list_len = lambda x: len(x)

df['RecentDelays_listlen'] = df['RecentDelays'].apply(lambda_list_len)
max_RecentDelays_len = df['RecentDelays_listlen'].max()

print("Maximum number of elements in the RecentDelays list is {}".format(max_RecentDelays_len))

Maximum number of elements in the RecentDelays list is 3


In [17]:
# initialise a dictionary to create the resulting datframe
dict_data = {}
for i in range(max_RecentDelays_len):
    dict_data[i] = []
    
print(dict_data)

{0: [], 1: [], 2: []}


In [18]:
#Function to update the dictionary with appropriate values
def getDelays(delaylst):
    for i in range(max_RecentDelays_len):
        if len(delaylst) > i: 
            dict_data[i].append(delaylst[i])
        else:
            dict_data[i].append('NaN')
    
df['RecentDelays'].apply(getDelays)

0    None
1    None
2    None
3    None
4    None
Name: RecentDelays, dtype: object

In [19]:
dict_data

{0: [23, 'NaN', 24, 13, 67],
 1: [47, 'NaN', 43, 'NaN', 32],
 2: ['NaN', 'NaN', 87, 'NaN', 'NaN']}

In [20]:
 pd.DataFrame(dict_data)

,0,1,2
0,23,47,NaN
1,NaN,NaN,NaN
2,24,43,87
3,13,NaN,NaN
4,67,32,NaN


### Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [21]:
delays = pd.DataFrame(dict_data)

In [22]:
delay_columns = []
for i in range(1,max_RecentDelays_len+1):
    delay_columns.append("delay_" + str(i))

print(delay_columns)

['delay_1', 'delay_2', 'delay_3']


In [23]:
delays.columns = delay_columns

In [24]:
delays

,delay_1,delay_2,delay_3
0,23,47,NaN
1,NaN,NaN,NaN
2,24,43,87
3,13,NaN,NaN
4,67,32,NaN


In [25]:
df = pd.concat([df, delays], axis=1)

In [26]:
df

,FlightNumber,RecentDelays,Airline,From,To,RecentDelays_listlen,delay_1,delay_2,delay_3
0,10045,"[23, 47]",KLM(!),London,Paris,2,23,47,NaN
1,10055,[],<Air France> (12),Madrid,Milan,0,NaN,NaN,NaN
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm,3,24,43,87
3,10075,[13],12. Air France,Budapest,Paris,1,13,NaN,NaN
4,10085,"[67, 32]","""Swiss Air""",Brussels,London,2,67,32,NaN


In [27]:
df.rename(columns = {'RecentDelays': 'Delays'}, inplace=True)

In [28]:
df

,FlightNumber,Delays,Airline,From,To,RecentDelays_listlen,delay_1,delay_2,delay_3
0,10045,"[23, 47]",KLM(!),London,Paris,2,23,47,NaN
1,10055,[],<Air France> (12),Madrid,Milan,0,NaN,NaN,NaN
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm,3,24,43,87
3,10075,[13],12. Air France,Budapest,Paris,1,13,NaN,NaN
4,10085,"[67, 32]","""Swiss Air""",Brussels,London,2,67,32,NaN
